In [1]:
import sys
import os
import hydra
from Config.config import Configuration
from AutoencoderU import AutoEncoder
import pytorch_lightning as L
from DataModule import ButterflyDataModule
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import wandb

wandb.login()

notebook_dir = os.path.dirname(os.path.abspath(os.getcwd()))
config_dir = os.path.join(notebook_dir,"proyecto-transfer-learning\\Config")
print(config_dir)

sys.argv = [arg for arg in sys.argv if not arg.startswith("--")]

@hydra.main(config_path=config_dir, config_name="config", version_base=None)
def trainining_autoencoder_ninety_10(config: Configuration):
    data_manager = ButterflyDataModule(config.DATASET.DATA_DIR,config.TRAIN.BATCH_SIZE, config.SPLIT.NEEDS_SPLIT, 
    config.SPLIT.SPLIT_RATIO,config.TRAIN.NUM_WORKERS)
    data_manager.setup()
    wandb_logger = WandbLogger(
        project="project-three-autoencoder",
        log_model=True,
    )
    autoencoder = AutoEncoder(config.TRAIN.LEARNING_RATE,config.MODEL.LATENT_DIM)
    print(type(autoencoder)) 
    trainer = L.Trainer(max_epochs=config.TRAIN.NUM_EPOCHS, accelerator=config.TRAIN.ACCELERATION,
    precision=config.TRAIN.PRECISION,callbacks=[EarlyStopping(monitor="autoencoder_val_loss", mode="min")], 
    logger=wandb_logger)
    trainer.fit(autoencoder,train_dataloaders=data_manager.unsupervised_train_loader(), 
    val_dataloaders=data_manager.test_dataloader())

    wandb_logger.experiment.finish()

trainining_autoencoder_ninety_10()


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: r29leonc (iaredescnncnn-instituto-tecnol-gico-de-costa-rica). Use `wandb login --relogin` to force relogin


c:\Users\richa\OneDrive\Desktop\TEC\proyecto-transfer-learning\Config
<class 'AutoencoderU.AutoEncoder'>


Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | encoder | Encoder | 4.7 M  | train
1 | decoder | Decoder | 3.0 M  | train
2 | loss_fn | MSELoss | 0      | train
--------------------------------------------
7.7 M     Trainable params
0         Non-trainable params
7.7 M     Total params
30.790    Total estimated model params size (MB)
42        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Error executing job with overrides: []
Traceback (most recent call last):
  File "C:\Users\richa\AppData\Local\Temp\ipykernel_20340\753413832.py", line 34, in trainining_autoencoder_ninety_10
    trainer.fit(autoencoder,train_dataloaders=data_manager.unsupervised_train_loader(),
  File "c:\Users\richa\anaconda3\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 538, in fit
    call._call_and_handle_interrupt(
  File "c:\Users\richa\anaconda3\Lib\site-packages\pytorch_lightning\trainer\call.py", line 47, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\richa\anaconda3\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 574, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "c:\Users\richa\anaconda3\Lib\site-packages\pytorch_lightning\trainer\trainer.py", line 981, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "c:\Users\richa\anaconda3\Lib\site-pac

AttributeError: 'tuple' object has no attribute 'tb_frame'